# Расчет СМО M/H2/n с H2-разогревом

### О режиме разогрева
Режим «разогрева» активизируется в случае, если система была пустой (на обслуживании и в очереди не было ни одной заявки) и прибыла очередная заявка. В этом случае обслуживание прибывшей заявки начнется после случайного периода «разогрева» системы.
Кроме того, все прибывающие во время «разогрева» системы заявки попадают в очередь и также не обслуживаются до окончания данного периода.

### Об алгоритме
В данной реализации предполагается, что время обслуживания и время разогрева задано *H2*-распределениями, каждое со своими параметрами. Параметры *H2*-распределений подбираются по заданным средним и коэффициентам вариации.

Алгорим позволяет рассчитать систему при произвольном коэффициенте вариации времени обслуживания. При коэффициенте вариации времени обслуживания < 1 параметры аппроксимирующего *Н2*-распределения являются комплексными, что не мешает получению осмысленных результатов.

Для верификации используется имитационное моделирование (ИМ)



### Задаем входные параметры:

In [1]:
from most_queue.theory.m_h2_h2warm import Mh2h2Warm
from most_queue.sim.qs_sim import QueueingSystemSimulator
from most_queue.sim import rand_destribution as rd
from most_queue.utils.tables import probs_print, times_print

import math
import time

n = 5  # число каналов
l = 1.0  # интенсивность вх потока
ro = 0.7  # коэфф загрузки
b1 = n * ro  # ср время обслуживания
b1_warm = n * 0.2  # ср время разогрева
num_of_jobs = 1000000  # число обсл заявок ИМ
b_coev = 1.1  # коэфф вариации времени обсл
b_coev_warm = 1.3  # коэфф вариации времени разогрева
verbose = False  # не выводить пояснения при расчетах

### Задаем начальные моменты времени обслуживания

In [2]:
b = [0.0] * 3
alpha = 1 / (b_coev ** 2)
b[0] = b1
b[1] = math.pow(b[0], 2) * (math.pow(b_coev, 2) + 1)
b[2] = b[1] * b[0] * (1.0 + 2 / alpha)

### Задаем начальные моменты времени разогрева

In [3]:
b_w = [0.0] * 3
b_w[0] = b1_warm
alpha = 1 / (b_coev_warm ** 2)
b_w[1] = math.pow(b_w[0], 2) * (math.pow(b_coev_warm, 2) + 1)
b_w[2] = b_w[1] * b_w[0] * (1.0 + 2 / alpha)

### Запускаем ИМ

In [4]:
sim_start = time.process_time()

qs = QueueingSystemSimulator(n)  # экземпляр класса ИМ
qs.set_sources(l, 'M')  # задаем параметры вх потока (параметры, тип распределения)

gamma_params = rd.Gamma.get_mu_alpha(b)
qs.set_servers(gamma_params, 'Gamma')  # задаем параметры обслуживания

gamma_params_warm = rd.Gamma.get_mu_alpha(b_w)
qs.set_warm(gamma_params_warm, 'Gamma')  # задаем параметры разогрева

qs.run(num_of_jobs)  # запуск ИМ

# после окончания ИМ сохраняем вероятности состояний и нач моменты времени пребывания
p = qs.get_p()
v_sim = qs.v

sim_time = time.process_time() - sim_start

Start simulation



100%|██████████| 3000000/3000000 [00:16<00:00, 183616.91it/s]


Simulation is finished



### Запуск расчета итерационным методом

In [5]:
tt_start = time.process_time()
tt = Mh2h2Warm(l, b, b_w, n, verbose=verbose)  # передаем параметры в экземпляр класса

tt.run()  # запуск расчета

# после окончания расчета сохраняем вероятности состояний и нач моменты времени пребывания
p_tt = tt.get_p()
v_tt = tt.get_v()

tt_time = time.process_time() - tt_start

num_of_iter = tt.num_of_iter_  # число итераций

D:\python\mps_14_08\venv\lib\site-packages\most_queue\theory\m_h2_h2warm.py:314: ComplexWarning: Casting complex values to real discards the imaginary part
  while math.fabs(1.0 - p_sum) > 1e-6:


### Вывод результатов

In [6]:
print("\nСравнение результатов расчета методом Такахаси-Таками и ИМ.\n"
      f"ИМ - M/Gamma/{n:^2d} с Gamma разогревом\n"
      f"Такахаси-Таками - M/H2/{n:^2d} c H2-разогревом"
      f"Коэффициент загрузки: {ro:^1.2f}")
print(f'Коэффициент вариации времени обслуживания {b_coev:0.3f}')
print(f'Коэффициент вариации времени разогрева {b_coev_warm:0.3f}')
print(f"Количество итераций алгоритма Такахаси-Таками: {num_of_iter:^4d}")
print(f"Время работы алгоритма Такахаси-Таками: {tt_time:^5.3f} c")
print(f"Время ИМ: {sim_time:^5.3f} c")

probs_print(p, p_tt, 10)

times_print(v_sim, v_tt, is_w=False)


Сравнение результатов расчета методом Такахаси-Таками и ИМ.
ИМ - M/Gamma/5  с Gamma разогревом
Такахаси-Таками - M/H2/5  c H2-разогревомКоэффициент загрузки: 0.70
Коэффициент вариации времени обслуживания 1.100
Коэффициент вариации времени разогрева 1.300
Количество итераций алгоритма Такахаси-Таками:  26 
Время работы алгоритма Такахаси-Таками: 0.109 c
Время ИМ: 7.047 c
------------------------------------
       Вероятности состояний        
------------------------------------
 №  |     Числ      |      ИМ       
------------------------------------
 0  |   0.021523    |   0.021702    
 1  |   0.086511    |   0.087107    
 2  |    0.15644    |    0.15634    
 3  |    0.18445    |    0.18424    
 4  |    0.16128    |    0.16112    
 5  |    0.11115    |    0.10992    
 6  |   0.078321    |   0.078089    
 7  |   0.055874    |   0.055749    
 8  |   0.040129    |   0.040355    
 9  |   0.028924    |   0.029204    
------------------------------------

Начальные моменты времени пребыв